In [ ]:
import numpy as np
import os, sys
import tqdm
from tqdm import trange

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # mps does not work


In [ ]:
import timm

def _create_model(model_name, classes):
    model = timm.create_model(MODEL_NAME, pretrained=True) # TODO Consider pretrained=True and False
    in_feats = model.fc.in_features
    model.fc = nn.Linear(in_feats, classes) # adapt final layer to match the 10 classes of CIFAR-10

    return model

MODEL_NAME = "resnet18.a1_in1k"
model = _create_model(MODEL_NAME, 10)

# import the model, training routines you need from your victim's code...
model_path = "./state_dicts/poison_resnet18.a1_in1k_cifar10_model.pt"
state_dict = torch.load(model_path)
model.load_state_dict(state_dict['model_state_dict'])

optimizer = state_dict['optimizer_state_dict']
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4, nesterov=True) # from poisoning example

loss_fn = torch.nn.CrossEntropyLoss()

model_art = PyTorchClassifier(
    model=model,
    loss=loss_fn,
    optimizer=optimizer,
    clip_values=(min,max),#TODO
    epsilon=0.3, #TODO
    input_shape=(1, 28, 28),
    nb_classes=10,
) # TODO 



In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10()

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

x_train = np.transpose(x_train, [0, 3,1,2])
x_test = np.transpose(x_test, [0, 3,1,2])

from art.utils import to_categorical

# A trigger from class 0 will be classified into class 1.
class_source = 0
class_target = 1
index_target = np.where(y_test.argmax(axis=1)==class_source)[0][5]

# Trigger sample
x_trigger = x_test[index_target:index_target+1]
y_trigger  = to_categorical([class_target], nb_classes=10)

In [ ]:
from art.attacks.poisoning.gradient_matching_attack import GradientMatchingAttack

epsilson = 0.01/(std+1e-7)

attack = GradientMatchingAttack(model_art,
        percent_poison=0.10,
        max_trials=1,
        max_epochs=100,
        clip_values=(min_,max_),
        learning_rate_schedule=(np.array([1e-1, 1e-2, 1e-3, 1e-4, 1e-5]), [250, 350, 400, 430, 460]),
        epsilon=epsilson,
        verbose=1)

x_poison, y_poison = attack.poison(x_trigger, y_trigger, x_train, y_train)

In [ ]:
#TODO: neues Modell trainieren mit poisonierten Daten trainieren
#TODO: neues Modell testen